In [32]:
import requests
from bs4 import BeautifulSoup
from datetime import date
import json
import types
from os import listdir
from os.path import isfile, join
from unidecode import unidecode
import pandas as pd
import codecs

In [33]:
LYRICSSITE = 'http://www.lyrics.wikia.com/api.php'
DATA_PATH = '../data/'
BILLBOARD_RAW_PATH = 'billboard_raw/'
BILLBOARD_LYRIC_PATH = 'billboard_lyrics/'
NUMBER, ARTIST, SONG = range(3)

In [56]:
def replace_br(elem,lyrics_soup):
    lyric = ''
    for br in lyrics_soup.findAll('br'):
        next_s = br.next_sibling
        if next_s:
            next2_s = next_s.next_sibling
        text = str(next_s).strip()
        if "<" in text:
            pass
        else:
            lyric = " ".join([lyric,text])
    return lyric

In [59]:
def download_lyrics(y):
    file = str(y) + 'hot100.csv'
    print(DATA_PATH+BILLBOARD_RAW_PATH+file)
    
    #loading raw billbaord file
    df = pd.read_csv(DATA_PATH+BILLBOARD_RAW_PATH+file,sep='@')
    rank_list = df['rank'].tolist()
    artist_list = df['artist'].tolist()
    title_list = df['title'].tolist()
    lyric_list = []
    
    for i in range(len(artist_list)):
        lyric_list.append("no lyrics") 
        params = {
            'action': 'lyrics',
            'artist': artist_list[i],
            'song': title_list[i]
        }
        link_r = requests.get(LYRICSSITE, params = params)
        link_soup = BeautifulSoup(link_r.text, 'html.parser')
        link = 0
        anc = link_soup.a
        if link_soup.a:
            if link_soup.a.has_attr('href'):
                link = anc['href']
        if link != 0:
            lyrics_r = requests.get(link)
            lyrics_soup = BeautifulSoup(lyrics_r.text, 'html.parser')
            lyrics_divs = lyrics_soup.findAll('div', { 'class': 'lyricbox'})

            for div in lyrics_divs:
                lyric = replace_br(div,lyrics_soup)
                if not lyric:
                    print('problem with lyric')
                lyric_list[-1] = lyric
        #Reference: https://github.com/alexmwu/hot100
        else:
            lyric_list.append('no lyrics')
            print('no lyric')

    d = {'rank': rank_list, 'title': title_list, 'artist': artist_list, 'lyric':lyric_list}
    df = pd.DataFrame(data=d)
    df = df[['rank','title','artist','lyric']]
    display(df)
    df.to_csv(DATA_PATH+BILLBOARD_LYRIC_PATH+file, sep='@') 

In [60]:
columns = ['position','year','title.href','title','artist','lyrics']

start_year = 2006
curr_year = date.today().year

for y in range(start_year, curr_year):
    download_lyrics(y)

../data/billboard_raw/2006hot100.csv


,rank,title,artist,lyric
0,1,Bad Day,Daniel Powter,"August 23, 2009 was a You kick up the leaves ..."
1,2,Temperature,Sean Paul,"was a So me give it to, so me give to, so me ..."
2,3,Promiscuous,Nelly Furtado Featuring Timbaland,was a Timbaland: Nope Nelly: Didn't think so ...
3,4,You're Beautiful,James Blunt,"February 11, 2007 was a My life is brilliant ..."
4,5,Hips Don't Lie,Shakira Featuring Wyclef Jean,"was a Ladies up in here tonight No fighting, ..."
5,6,Unwritten,Natasha Bedingfield,"April 05, 2007 was a Can't read my mind I'm u..."
6,7,Crazy,Gnarls Barkley,"was a I remember, I remember when I lost my m..."
7,8,Ridin',Chamillionaire Featuring Krayzie Bone,"August 10, 2006 was a Patrollin' and tryin' t..."
8,9,SexyBack,Justin Timberlake,was a Them other boys don't know how to act I...
9,10,Check On It,Beyonce Featuring Slim Thug,was a With all them clowns and the wangstas G...


../data/billboard_raw/2007hot100.csv


,rank,title,artist,lyric
0,1,Irreplaceable,Beyonce,"January 01, 2007 was a To the left, to the le..."
1,2,Umbrella,Rihanna Featuring Jay-Z,was a Uh-huh uh-huh (Good girl gone bad) Uh-h...
2,3,The Sweet Escape,Gwen Stefani Featuring Akon,"was a Woo-hoo, yee-hoo Woo-hoo, yee-hoo Woo-h..."
3,4,Big Girls Don't Cry,Fergie,was a The smell of your skin lingers on me no...
4,5,Buy U A Drank (Shawty Snappin'),T-Pain Featuring Yung Joc,"was a T-Pain Damn, shawty snap Yung Joc (Shaw..."
5,6,Before He Cheats,Carrie Underwood,was a And she's probably getting frisky Right...
6,7,Hey There Delilah,Plain White T's,"April 24, 2007 was a What's it like in New Yo..."
7,8,I Wanna Love You,Akon Featuring Snoop Dogg,"was a Convict, music And you know we up front..."
8,9,Say It Right,Nelly Furtado,"June 04, 2007 was a Say it right, say it all ..."
9,10,Glamorous,Fergie Featuring Ludacris,was a If you ain't got no money take yo' brok...


../data/billboard_raw/2008hot100.csv


...

,rank,title,artist,lyric
0,1,Uptown Funk!,Mark Ronson Featuring Bruno Mars,"was a This hit, that ice cold Michelle Pfeiff..."
1,2,Thinking Out Loud,Ed Sheeran,"November 30, 2014 was a And I can't sweep you..."
2,3,See You Again,Wiz Khalifa Featuring Charlie Puth,was a And I'll tell you all about it when I s...
3,4,Trap Queen,Fetty Wap,"was a 1738 I'm like ""hey, what's up, hello"" S..."
4,5,Sugar,Maroon 5,"was a I need your loving, loving, I need it n..."
5,6,Shut Up And Dance,WALK THE MOON,"was a Just keep your eyes on me"" I said, ""You..."
6,7,Blank Space,Taylor Swift,was a I could show you incredible things Magi...
7,8,Watch Me,Silento,no lyrics
8,9,Earned It (Fifty Shades Of Grey),The Weeknd,"was a 'Cause I see nobody, nobody but you, yo..."
9,10,The Hills,The Weeknd,was a You said keep our business on the low-l...


../data/billboard_raw/2016hot100.csv


,rank,title,artist,lyric
0,1,Love Yourself,Justin Bieber,was a And all the clubs you get in using my n...
1,2,Sorry,Justin Bieber,was a You know I try but I don't do too well ...
2,3,One Dance,Drake Featuring WizKid & Kyla,"was a Grips on your waist Front way, back way..."
3,4,Work,Rihanna Featuring Drake,"was a You see me I be Work, work, work, work,..."
4,5,Stressed Out,twenty one pilots,was a I wish I had a better voice that sang s...
5,6,Panda,Desiigner,was a I guess so They been waiting for this s...
6,7,Hello,Adele,"March 20, 2018 January 22, 2018 was a I was w..."
7,8,Don't Let Me Down,The Chainsmokers Featuring Daya,was a Right now I need a miracle Hurry up now...
8,9,Can't Stop The Feeling!,Justin Timberlake,"was a It goes electric, wavey when I turn it ..."
9,10,Closer,The Chainsmokers Featuring Halsey,was a I drink too much and that's an issue bu...


../data/billboard_raw/2017hot100.csv


,rank,title,artist,lyric
0,1,Shape Of You,Ed Sheeran,was a So the bar is where I go Me and my frie...
1,2,Despacito,Luis Fonsi & Daddy Yankee Featuring Justin Bieber,"So thankful for that, it's such a blessin', y..."
2,3,That's What I Like,Bruno Mars,"was a Baby girl, what's hatnin'? You and your..."
3,4,Humble.,Kendrick Lamar,"was a Even a day for me Way (yeah, yeah) Aye,..."
4,5,Something Just Like This,The Chainsmokers & Coldplay,was a The legends and the myths Achilles and ...
5,6,Bad And Boujee,Migos Featuring Lil Uzi Vert,was a You know so we never really had no old ...
6,7,Closer,The Chainsmokers Featuring Halsey,was a I drink too much and that's an issue bu...
7,8,Body Like A Back Road,Sam Hunt,"was a First time I seen her walk by, man I 'b..."
8,9,Believer,Imagine Dragons,was a I'mma say all the words inside my head ...
9,10,Congratulations,Post Malone Featuring Quavo,was a Yeah-yeah No-nah-nah Yeah My mama calle...
